In [4]:
import matplotlib.pyplot as plt
import numpy as np
from ctypes import *
import numpy as np
from PIL import Image, ImageOps
import os
from tqdm import tqdm
tqdm.pandas()
import pickle
from sklearn.model_selection import train_test_split

import des fonctions de la lib python pour le modele MLP

In [2]:
path_to_shared_library ="./CPPDLLForPython/cmake-build-debug/CPPDLLForPython.dll"
my_lib = cdll.LoadLibrary(path_to_shared_library)

In [37]:
def create_MLP_model(npl, dsize):
    npltype = POINTER(c_int)
    dsizetype = c_int
    
    my_lib.create_MLP_model.argtypes = [npltype, dsizetype]
    my_lib.create_MLP_model.restypes = POINTER(c_void_p)

    model = my_lib.create_MLP_model(npl, dsize)
    return model

def train_classification_stochastic_backprop_mlp_model(model, flattened_dataset_inputs,flattened_expected_outputs,alpha = 0.01,iterations_count = 1000):
    model_type = POINTER(c_void_p)
    flattened_dataset_inputs_type = POINTER(c_void_p)
    flattened_expected_outputs_type = POINTER(c_void_p)

    my_lib.predict_linear_model_regression.argtypes = [model_type, flattened_dataset_inputs_type, flattened_expected_outputs_type, c_float, c_int]
    my_lib.predict_linear_model_regression.restypes = None

def train_regression_stochastic_backprop_mlp_model(model, flattened_dataset_inputs,flattened_expected_outputs,alpha = 0.01,iterations_count = 1000):
    model_type = POINTER(c_void_p)
    flattened_dataset_inputs_type = POINTER(c_void_p)
    flattened_expected_outputs_type = POINTER(c_void_p)

    my_lib.predict_linear_model_regression.argtypes = [model_type, flattened_dataset_inputs_type, flattened_expected_outputs_type, c_float, c_int]
    my_lib.predict_linear_model_regression.restypes = None

def predict_mlp_model_classification(model, sample_inputs):
    model_type = POINTER(c_void_p)
    sample_inputs_type = POINTER(c_void_p)

    my_lib.predict_mlp_classification.argtypes = [model_type, sample_inputs_type]
    my_lib.predict_mlp_classification.restypes = c_float

    predict_model = my_lib.predict_mlp_model_classification(model, sample_inputs)

    return predict_model

def predict_mlp_model_regression(model, sample_inputs):
    model_type = POINTER(c_void_p)
    sample_inputs_type = POINTER(c_void_p)

    my_lib.predict_mlp_model_regression.argtypes = [model_type, sample_inputs_type]
    my_lib.predict_mlp_model_regression.restypes = c_float

    predict_model = my_lib.predict_mlp_model_regression(model, sample_inputs)

    return predict_model

Import des images pour le dataset et creation de methode de preprocessing de données

In [5]:
def preprocess_dataset(path,img_size):
    list_X,list_Y = [],[]
    for filename in tqdm(os.listdir(path)):
        img = ImageOps.grayscale(Image.open(path+"/"+filename).resize(img_size))
        array = np.asarray(img).flatten().tolist()
        
        list_X.append(array)
    
        if "Dataset_Mangas" in path:
            class_array = 0
        elif "Dataset_Romans" in path:
            class_array = 1
        elif "Dataset_Comics" in path:
            class_array = 2
        
        list_Y.append(class_array)
        
      
    X ,Y = np.array(list_X), np.array(list_Y)
    return X/255.**2,Y.reshape((Y.shape[0],))/255.**2

In [6]:
X_mangas,Y_mangas = preprocess_dataset("C:/Users/Mamadian/Documents/ESGI/MachineLearning/PA_ML_3IABD/dataset/Dataset_Mangas/",(56,56))

100%|██████████| 2034/2034 [01:36<00:00, 21.06it/s]


In [7]:
X_comics,Y_comics = preprocess_dataset("C:/Users/Mamadian/Documents/ESGI/MachineLearning/PA_ML_3IABD/dataset/Dataset_Comics/",(56,56))

100%|██████████| 2089/2089 [03:55<00:00,  8.87it/s]


In [8]:
X_romans,Y_romans = preprocess_dataset("C:/Users/Mamadian/Documents/ESGI/MachineLearning/PA_ML_3IABD/dataset/Dataset_Romans/",(56,56))

100%|██████████| 2571/2571 [02:45<00:00, 15.57it/s]


In [11]:
X_mangas_train, X_mangas_test, y_mangas_train, y_mangas_test = train_test_split(X_mangas, Y_mangas, test_size=0.33, random_state=42)
X_romans_train, X_romans_test, y_romans_train, y_romans_test = train_test_split(X_romans, Y_romans, test_size=0.33, random_state=42)
X_comics_train, X_comics_test, y_comics_train, y_comics_test = train_test_split(X_comics, Y_comics, test_size=0.33, random_state=42)

In [12]:
X_train = np.concatenate((X_mangas_train, X_romans_train, X_comics_train), axis=0)
y_train = np.concatenate((y_mangas_train, y_romans_train, y_comics_train), axis=0)

X_test = np.concatenate((X_mangas_test, X_romans_test, X_comics_test), axis=0)
y_test = np.concatenate((y_mangas_test, y_romans_test, y_comics_test), axis=0)

In [15]:
X_train.shape

(4483, 3136)

In [38]:
pyarr = [2,3,1]
tab = (c_int*len(pyarr))(*pyarr)
mlp = create_MLP_model(tab,3)

In [39]:
type(mlp)

int

In [17]:
train_classification_stochastic_backprop_mlp_model(mlp, X_train, y_train, alpha = 0.01, iterations_count = 1000)

In [20]:
def process_image(filepath, img_size):
    
    img = ImageOps.grayscale(Image.open(filepath).resize(img_size))
    array = np.asarray(img) / 255**2
    
    return np.array(array.flatten().tolist()).reshape((1,img_size[0]*img_size[1]))

In [31]:
img = process_image("C:/Users/Mamadian/Documents/ESGI/MachineLearning/PA_ML_3IABD/dataset/AT_WildHunt_Final-04-flat.jpg",(56,56))
img

array([[0.00392157, 0.00392157, 0.00392157, ..., 0.00392157, 0.00392157,
        0.00392157]])

In [40]:
res = predict_mlp_model_regression(mlp, img)

ArgumentError: argument 1: <class 'TypeError'>: expected LP_c_void_p instance instead of int